# Imports and installations

In [ ]:
!pip install cartopy

In [ ]:
# !pip install netCDF4 h5netcdf

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm
from tqdm.auto import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Daily maximums caclulation


Here we are using masked data for defining HW days over Bangladesh. This masked data was created in
- `EPT & t2m _masking_for_BD.ipynb`

In [ ]:
# Taking the masked data for defining HW over BD only
ds_ept=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/ept_bdt_masked.nc')
ds_t2m=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/t2m_bdt_masked.nc')

In [ ]:
# ds_ept

In [ ]:
# ds_t2m

In [ ]:
ds_ept_daily_max=ds_ept.ept.resample(time='D').max()
ds_t2m_daily_max=ds_t2m.t2m.resample(time='D').max()



In [ ]:
# ds_ept_daily_max

In [ ]:
# ds_t2m_daily_max

# Rolling Threshold calculation


- To calculate the 95th percentile threshold for each calendar day, we applied a 15-day rolling window centered on that day for the period 1981-2010.
- This procedure was repeated for every calendar day and at each grid point.

In [ ]:
# Remove leap day (Feb 29)
max_t2m = ds_t2m_daily_max.sel(time=~((ds_t2m_daily_max.time.dt.month == 2) & (ds_t2m_daily_max.time.dt.day == 29)))
# max_t2m
max_ept = ds_ept_daily_max.sel(time=~((ds_ept_daily_max.time.dt.month == 2) & (ds_ept_daily_max.time.dt.day == 29)))
# max_ept

- Organizing data based on day of the year.

In [ ]:
# Select T2M data for years 1981–2010
max_t2m_copy = max_t2m.sel(time=max_t2m.time.dt.year.isin(range(1981,2011)))
# Add day-of-year (DOY) coordinate
max_t2m_copy['doy'] = max_t2m_copy['time'].dt.dayofyear
max_t2m_copy

# Select T2M data for years 1981–2010
max_ept_copy = max_ept.sel(time=max_ept.time.dt.year.isin(range(1981,2011)))
# Add day-of-year (DOY) coordinate
max_ept_copy['doy'] = max_ept_copy['time'].dt.dayofyear
max_ept_copy

- Applying the 15 day rolling window.

In [ ]:
# Loop through 351 days (sliding 15-day windows)

t2m_day_group=max_t2m_copy.groupby('doy')
t2m_percentile_list=[]
for i in range(351):
  # Combine data from a 15-day window
  combined = xr.concat([t2m_day_group[d] for d in range(i+1,i+16)], dim="time")
  # Calculate 95th percentile for each grid cell
  percentile_95 = combined.quantile(0.95, dim="time")
  # Assign corresponding DOY (centered at i+8)
  percentile_95 = percentile_95.assign_coords(doy=8+i)
  # Convert to dataset format
  percentile_95 = percentile_95.to_dataset()
  # Append to list
  t2m_percentile_list.append(percentile_95)

# Concatenate results into a single dataset along DOY dimension
t2m_percentile_doy = xr.concat(t2m_percentile_list, dim='doy')
# t2m_percentile_doy


# Group EPT by DOY
ept_day_group = max_ept_copy.groupby('doy')
ept_percentile_list = []

# Loop for sliding 15-day window
for i in range(351):
  # Combine 15 days
  combined = xr.concat([ept_day_group[d] for d in range(i+1,i+16)], dim="time")
  # 95th percentile
  percentile_95 = combined.quantile(0.95, dim="time")
  # Assign DOY
  percentile_95 = percentile_95.assign_coords(doy=8+i)
  # To dataset
  percentile_95 = percentile_95.to_dataset()
  # Append
  ept_percentile_list.append(percentile_95)

# Merge EPT percentiles
ept_percentile_doy = xr.concat(ept_percentile_list, dim='doy')
# ept_percentile_doy

In [ ]:
# Full DOY range 1–365
all_doys = np.arange(1, 366)

# Reindex the doy dimension to include all days (filling new ones with NaN)
t2m_expanded_ds = t2m_percentile_doy.reindex(doy=all_doys)
t2m_expanded_ds.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/t2m_percentile_doy.nc')

ept_expanded_ds = ept_percentile_doy.reindex(doy=all_doys)
ept_expanded_ds.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/ept_percentile_doy.nc')


In [ ]:
# ept_expanded_ds

# Data with threshold

In [ ]:
# Adding the threshold to each variable dataset to compare.

t2m_year_group=max_t2m.groupby('time.year')
t2m_yearly_list=[]
for year in range(1971,2025):
  percentile_array = xr.DataArray(
      data=t2m_expanded_ds.t2m.values,  # Ensure shape matches
      coords=t2m_year_group[year].coords,
      dims=t2m_year_group[year].dims
  )
  # Add new variable
  year_group_ds = t2m_year_group[year].to_dataset()  # Convert to Dataset
  year_group_ds["95th_t2m"] = percentile_array  # Assign the new array
  t2m_yearly_list.append(year_group_ds)
t2m_max_data=xr.concat(t2m_yearly_list,dim='time')
# t2m_max_data

ept_year_group=max_ept.groupby('time.year')
ept_yearly_list=[]
for year in range(1971,2025):
  percentile_array = xr.DataArray(
      data=ept_expanded_ds.ept.values,  # Ensure shape matches
      coords=ept_year_group[year].coords,
      dims=ept_year_group[year].dims
  )
  # Add new variable
  year_group_ds = ept_year_group[year].to_dataset()  # Convert to Dataset
  year_group_ds["95th_ept"] = percentile_array  # Assign the new array
  ept_yearly_list.append(year_group_ds)
ept_max_data=xr.concat(ept_yearly_list,dim='time')
# ept_max_data

In [ ]:
print(ept_expanded_ds.ept.shape,ept_year_group[1971].shape)

# Warm days (days surpassing the thresholds)

In [ ]:
t2m_march_september=t2m_max_data.sel(time=t2m_max_data.time.dt.month.isin([3,4,5,6,7,8,9]))
ept_march_september=ept_max_data.sel(time=ept_max_data.time.dt.month.isin([3,4,5,6,7,8,9]))

# # t2m_march_september
# # ept_march_september

In [ ]:
df_t2m=t2m_march_september.to_dataframe().reset_index()
df_ept=ept_march_september.to_dataframe().reset_index()


In [ ]:
# df_t2m

In [ ]:
# df_ept

In [ ]:
df = pd.merge(df_t2m,df_ept)
df

In [ ]:
df['warm_days']=0
# df

In [ ]:
 # checking the threshold conditions

hw_condition= (df['ept']>df['95th_ept'])|(df['t2m']>df['95th_t2m'])
df.loc[hw_condition,'warm_days']=1


In [ ]:
# df[df['warm_days']==1]


In [ ]:
df['warm_days'].value_counts()


## Heatwave days List

In [ ]:
df['heatwave']=df['warm_days']
# df

In [ ]:
df['heatwave'].value_counts()

In [ ]:
df['time']=pd.to_datetime(df['time'])
dates=df.time.unique()

In [ ]:
dates

In [ ]:
##### Heatwave Definition: minimum 3 consecutive days and 75 grids spread over BD #####

iteration = 0
starting_date = []
duration = []
ending_date = []
temporary_list = []

# Loop through all dates
for i in tqdm(range(len(dates)-1), desc='Dates', leave=True):
    # Count grids marked as warm days
    grid_count = (df[df['time'] == dates[i]]['warm_days'].values == 1).sum()
    # Check if current day is consecutive
    consecutive_day = (dates[i] - dates[i-1]).days == 1

    # If condition met: at least 75 grids and consecutive day
    if grid_count >= 75 and consecutive_day:
        iteration += 1
        temporary_list.append(dates[i])
    else:
        # If <3 days, reset heatwave flag
        if iteration <= 2:
            condition_1 = (df['time'] >= dates[i-iteration]) & (df['time'] <= dates[i])
            df.loc[condition_1, 'heatwave'] = 0
        # If ≥3 days, record as heatwave
        if iteration >= 3:
            starting_date.append(temporary_list[0])
            ending_date.append(temporary_list[-1])
            duration.append(iteration)
            condition_3 = (df['time'] == dates[i])
            df.loc[condition_3, 'heatwave'] = 0
        # Reset counters
        iteration = 0
        temporary_list = []

# Print counts of detected events
print(len(starting_date), len(ending_date), len(duration))

##### end #####



In [ ]:
df = pd.DataFrame({'Starting_Date': starting_date, 'Ending_Date': ending_date,'Duration':duration})
df

In [ ]:
# this is the dataframe for heatwave dates.
df = df.to_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv',index=False)
df

## Import the file to plot

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
df

In [ ]:
df['Starting_Date'] = pd.to_datetime(df["Starting_Date"])
df['Ending_Date'] = pd.to_datetime(df["Ending_Date"])
df[df['Starting_Date'].dt.year == 2023]

In [ ]:
# Calculate the time difference and store in a new column as the difference in days
# we are gonna merge any hw events separated by 2 days
df['Next_Row_Difference'] = (df['Starting_Date'].shift(-1) - df['Ending_Date']).dt.days
df=df.fillna(0)
# df

In [ ]:
# Create a new column to group rows where the difference is <= 2 or having 1 day gap
df['Flag'] = (~(df['Next_Row_Difference'] <=2))
# df

In [ ]:
# Initialize the number counter
current_number = 1

# Iterate through the 'Flag' column
for i, value in enumerate(df['Flag']):
    if value:  # For True values
        df.at[i, 'Group'] = current_number
        current_number += 1  # Increment the number for each True
    else:  # For False values
        df.at[i, 'Group'] = current_number

In [ ]:
# df[(df['Starting_Date'].dt.year == 2023) | (df['Starting_Date'].dt.year == 2021)]

In [ ]:
# Merge rows in each group
df_merged = df.groupby('Group', as_index=False).agg({
    'Starting_Date': 'first',  # Take the first starting date in the group
    'Ending_Date': 'last',  # Take the last ending date in the group
})

# View the resulting merged DataFrame
df_merged.drop('Group',axis=1, inplace=True)


In [ ]:
df_merged['Duration']=(df_merged['Ending_Date'] - df_merged['Starting_Date']).dt.days +1
df_merged

In [ ]:
df_merged[df_merged['Starting_Date'].dt.year == 2021]

# Calculation by season

In [ ]:
df= df_merged
df['Starting_Date']=pd.to_datetime(df["Starting_Date"])
df['Year']=df['Starting_Date'].dt.year
yearly_sum=df.groupby('Year')['Duration'].sum().reset_index()

grouped = df.groupby('Year')
yearly_event_count=list(grouped.size())
len(yearly_event_count)


yearly_sum['total_events'] = yearly_event_count

pre_monsoon=df[(df['Starting_Date'].dt.month >= 3) & (df['Starting_Date'].dt.month <= 6)]
# pre_monsoon

yearly_pre_monsoon=pre_monsoon.groupby('Year')['Duration'].sum().reset_index()

all_years=range(1971,2024)
missing_years=set(all_years)-set(yearly_pre_monsoon['Year'])
missing_years_df=pd.DataFrame({'Year':list(missing_years),'Duration':0,'total_events':0})
yearly_pre_monsoon=pd.concat([yearly_pre_monsoon,missing_years_df],ignore_index=True)
yearly_pre_monsoon=yearly_pre_monsoon.sort_values(by='Year').reset_index(drop=True)

#Monsoon
moonson=df[(df['Starting_Date'].dt.month>=7)&(df['Starting_Date'].dt.month<=9)]
yearly_moonson=moonson.groupby('Year')['Duration'].sum().reset_index()
all_years=range(1971,2024)
missing_years=set(all_years)-set(yearly_moonson['Year'])
missing_years_df=pd.DataFrame({'Year':list(missing_years),
                              'Duration':0,'total_events':0})
yearly_moonson=pd.concat([yearly_moonson,missing_years_df],ignore_index=True)
yearly_moonson=yearly_moonson.sort_values(by="Year").reset_index(drop=True)




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal.windows import gaussian
from scipy.ndimage import convolve1d
# Create a Gaussian kernel
window_size = 20  # Length of the kernel
sigma = 3.5
gaussian_kernel = gaussian(window_size, std=sigma)
gaussian_kernel /= gaussian_kernel.sum()  # Normalize kernel
gaussian_kernel

## Plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal.windows import gaussian
from scipy.ndimage import convolve1d

# Specify the size of the figure (width, height) in inches
fig, ax = plt.subplots(figsize=(10, 4))

# Assuming yearly data is available
tickname = list(yearly_moonson['Year'])

x = np.arange(len(tickname))  # Create indices for the x-axis
y = np.arange(60)

y1 = yearly_pre_monsoon['Duration']
y2 = yearly_moonson['Duration']

#  Gaussian kernel
window_size = 20  # Length of the kernel
sigma = 3.5
gaussian_kernel = gaussian(window_size, std=sigma)
gaussian_kernel /= gaussian_kernel.sum()  # Normalize kernel

# Gaussian smoothing
smooth_y1 = convolve1d(y1, gaussian_kernel, mode='reflect')
smooth_y2 = convolve1d(y2, gaussian_kernel, mode='reflect')

# Plot the original data
ax.plot(tickname, y1, alpha=0.3, color='red')
ax.plot(tickname, y2, alpha=0.3, color='purple')

# Plot the smoothed data
ax.plot(tickname, smooth_y1, color='red', linewidth=2, label='Pre-monsoon')
ax.plot(tickname, smooth_y2, color='purple', linewidth=2, label='Monsoon')

# Adding a vertical line at the year 1980 and 2023
baseline=2000
endline=2024
ax.axvline(x=baseline, color='gray', linestyle='--', linewidth=1.5)
ax.axvline(x=endline, color='gray', linestyle='--', linewidth=1.5)

# Adding intersection points
ax.scatter(baseline, smooth_y1[baseline-1971], color='red', s=40)
ax.scatter(endline, smooth_y1[endline-1971], color='red', marker='*', s=40)
ax.scatter(baseline, smooth_y2[baseline-1971], color='purple', s=40)
ax.scatter(endline, smooth_y2[endline-1971], color='purple', marker='*', s=40)

# Calculating the difference between 2023 and 2000
pre_monsoon_rise = smooth_y1[endline-1971] - smooth_y1[baseline-1971]
monsoon_rise = smooth_y2[endline-1971] - smooth_y2[baseline-1971]

# text to highlight trends
ax.text(2028, 32, f'+{pre_monsoon_rise} days', fontsize=12, color='red', ha='center', fontweight='bold')
ax.text(2028, 42, f'+{monsoon_rise} days', fontsize=12, color='purple', ha='center', fontweight='bold')
ax.text(2029.2, 52, 'Mean change', fontsize=12, color='black', ha='center', fontweight='bold')
ax.text(2029, 47, f'{endline} vs. {baseline}', fontsize=11, color='black', ha='center')

# Removing top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# xticks and labels with rotation
ax.set_xticks(tickname[::2])
ax.set_xticklabels(tickname[::2], rotation=45)
# xticks and labels with rotation
yticks=np.arange(0,56)
ax.set_yticks(yticks[::5])
ax.set_yticklabels(yticks[::5])

# Display the legend (only for smoothed lines)
ax.legend(loc='center',bbox_to_anchor=(0.5, 1.0), fontsize=10,ncol=2)
ax.grid(axis='y', linestyle='--')

plt.xlabel('Year')
plt.ylabel('Heatwave Days per year')
plt.title('Time Series and Trends of Heatwave Days by Season',y=1.05,fontsize=11)
plt.show()

# Save the figure
fig.savefig("/content/drive/MyDrive/AP_HW/Scripts-ll/Manuscript codes/All_Figures/Fig_4_time_series_of_heatwave_days.jpg",
            dpi=300, format="jpg", bbox_inches="tight")